005に対して以下の変更を入れた。

承知いたしました。「サーバーのグローバルアイテム埋め込みを直接更新」するのではなく、サーバー側でオプティマイザを使って「学習」させる形に修正を加えます。

この変更により、サーバーの`global_item_embedding`は、勾配降下法によって更新される通常のPyTorchモジュールのパラメータとして扱われるようになります。

**主な変更点:**

1.  **`Server` クラスにオプティマイザを追加**: `__init__` メソッドで `self.global_item_optimizer` を初期化します。

2.  **`aggregate_item_embeddings` 関数を修正**:

      * この関数は、グラフ畳み込みとFedAvgによる加重平均の結果として得られた `new_global_item_embedding_weight` を直接 `copy_` するのではなく、サーバーのオプティマイザが最適化するための「ターゲット」または「参照」として利用します。
      * **重要な概念**: Federated Learningにおけるサーバー側の更新は、ローカルモデルの平均化が「サーバーモデルの勾配」と見なされることが多いです。ここでは、`R_tensor` から得られた `new_global_item_embedding_weight` を、サーバーの`global_item_embedding`が目指すべき理想的な状態として扱い、その状態に近づくようにオプティマイザを動かす、という解釈を取ります。
      * ただし、PyTorchの通常の学習ループでは、損失を計算し、`backward()` を呼び出して勾配を計算し、`optimizer.step()` でパラメータを更新します。サーバー側での「学習」を厳密に再現するには、`new_global_item_embedding_weight` を用いて何らかの「損失」を定義し、それを最小化する形で`global_item_embedding`を更新する必要があります。

    最もシンプルで一般的なアプローチは、FedAvgにおける「サーバーモデルは、クライアントモデルの重み付き平均に更新される」という概念をPyTorchの最適化パスに変換することです。これは通常、以下のように行われます。

      * サーバーのオプティマイザの勾配をゼロにする。
      * サーバーの`global_item_embedding`の現在の重みと、クライアントから集約された新しいターゲット重みとの間の「差」を損失として扱う（例えば、MSEロス）。
      * この損失に基づいて勾配を計算し、オプティマイザでステップを実行する。

    しかし、論文の文脈では、この「学習」は必ずしも厳密な意味でのロス最小化を伴うとは限りません。FedAvgの多くは、単に重み付き平均を計算して直接適用します。そこで、今回は`optimizer.step()`を利用しつつ、**グラフ集約された結果をサーバーモデルが「学習」する**という解釈を強調するために、`new_global_item_embedding_weight`をターゲットとして、サーバーの埋め込みを更新する形にします。

    **より実践的なアプローチ**: グローバルアイテム埋め込みを直接最適化するには、集約された `new_global_item_embedding_weight` を「擬似的な勾配」として利用するか、あるいはそれを「ターゲット」として、現在の `self.global_item_embedding.weight` との差分を損失として最適化する方法が考えられます。後者の方法を採用します。




In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Subset
from transformers import AutoModel, AutoTokenizer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict

# 軽量 LLM 埋め込みモデルのロード (変更なし)
plm_model_name = "sentence-transformers/all-MiniLM-L6-v2"
plm_tokenizer = AutoTokenizer.from_pretrained(plm_model_name)
plm_model = AutoModel.from_pretrained(plm_model_name)

# PLMは学習済みモデルのため、勾配計算を無効化
for param in plm_model.parameters():
    param.requires_grad = False

plm_embedding_dim = plm_model.config.hidden_size
print(f"PLM embedding dimension: {plm_embedding_dim}")


class ClientModel(nn.Module):
    def __init__(self, num_items, item_embedding_dim, plm_model, plm_embedding_dim, joint_embedding_output_dim):
        super(ClientModel, self).__init__()
        self.plm_model = plm_model
        
        # Joint Embedding Layer (module parameter θ_user)
        self.user_joint_embedding_linear = nn.Linear(plm_embedding_dim, joint_embedding_output_dim)
        
        # Item Embedding Layer (module parameter θ_item)
        self.local_item_embedding = nn.Embedding(num_items, item_embedding_dim)

        # User Feature Refinement MLP (module parameter θ_umlp)
        self.user_mlp = nn.Sequential(
            nn.Linear(joint_embedding_output_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32)
        )

        # Predictive Scoring Function (module parameter θ_score)
        self.prediction_mlp = nn.Sequential(
            nn.Linear(item_embedding_dim + joint_embedding_output_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1)
        )

    def forward(self, user_ids, item_ids, user_texts_batch): 
        encoded_input = plm_tokenizer(user_texts_batch, padding=True, truncation=True, return_tensors='pt')
        plm_output = self.plm_model(**encoded_input).last_hidden_state[:, 0, :]

        user_raw_embedding = self.user_joint_embedding_linear(plm_output)
        user_embedding = self.user_mlp(user_raw_embedding)

        item_embedding = self.local_item_embedding(item_ids)

        combined_features = torch.cat((user_embedding, item_embedding), dim=1)
        logits = self.prediction_mlp(combined_features)
        predictions = torch.sigmoid(logits)

        return predictions, self.user_joint_embedding_linear.weight, self.local_item_embedding.weight


class Server:
    def __init__(self, num_users, num_items, item_embedding_dim, joint_embedding_output_dim):
        self.global_item_embedding = nn.Embedding(num_items, item_embedding_dim)
        # サーバーのグローバルアイテム埋め込みのためのオプティマイザ
        # 学習率 (lr) はクライアントとは別に設定可能
        self.global_item_optimizer = optim.Adam(self.global_item_embedding.parameters(), lr=0.01) # サーバー側の学習率

        self.num_users = num_users
        self.num_items = num_items
        self.item_embedding_dim = item_embedding_dim
        self.joint_embedding_output_dim = joint_embedding_output_dim
    
    def build_user_relationship_graph(self, user_linear_weights_map):
        sorted_user_ids = sorted(user_linear_weights_map.keys())
        if not sorted_user_ids:
            return np.zeros((0, 0)), []

        user_weight_vectors = np.array([
            user_linear_weights_map[u_id].cpu().numpy() for u_id in sorted_user_ids
        ])

        similarity_matrix = cosine_similarity(user_weight_vectors)
        user_graph_adj = similarity_matrix 
        
        return user_graph_adj, sorted_user_ids

    def update_global_item_embedding(self, user_local_item_weights, user_graph_adj, sorted_user_ids, client_data_sizes):
        """
        ユーザー関係グラフに基づいてアイテム埋め込みを集約し、サーバーのオプティマイザでグローバル埋め込みを学習・更新します。
        
        Args:
            user_local_item_weights (dict): {user_id: local_item_embedding.weight.data (Tensor)}
            user_graph_adj (np.ndarray): ユーザーグラフの隣接行列
            sorted_user_ids (list): user_graph_adj のノード順に対応するユーザーIDのリスト
            client_data_sizes (dict): {user_id: そのユーザーのデータセットサイズ}
            
        Returns:
            torch.Tensor: 更新されたグローバルアイテム埋め込みの重み
        """
        if not user_local_item_weights:
            return self.global_item_embedding.weight.data

        item_embedding_matrix_A = torch.stack([
            user_local_item_weights[u_id] for u_id in sorted_user_ids
        ]) # (num_users, num_items, item_embedding_dim)

        row_sums_graph = np.sum(user_graph_adj, axis=1, keepdims=True)
        row_sums_graph[row_sums_graph == 0] = 1 
        normalized_user_graph_adj = user_graph_adj / row_sums_graph
        
        normalized_user_graph_adj_tensor = torch.tensor(normalized_user_graph_adj, dtype=torch.float32)

        R_tensor = torch.einsum('ij, jkd -> ikd', normalized_user_graph_adj_tensor, item_embedding_matrix_A)

        # FedAvg: D = degree matrix at the time of aggregation
        total_data_size = sum(client_data_sizes[u_id] for u_id in sorted_user_ids)
        if total_data_size == 0:
            return self.global_item_embedding.weight.data 

        weighted_sum_item_embeddings = torch.zeros_like(self.global_item_embedding.weight) # .dataは使わない
        
        for i, u_id in enumerate(sorted_user_ids):
            weight = client_data_sizes[u_id] / total_data_size
            weighted_sum_item_embeddings += weight * R_tensor[i]

        # ここで、サーバーのグローバルアイテム埋め込みを学習させる
        self.global_item_optimizer.zero_grad()
        
        # 現在のグローバル埋め込みと目標値 (weighted_sum_item_embeddings) との間のロスを計算
        # MSELoss を使用して、目標値に近づけるように学習
        loss_fn = nn.MSELoss()
        # detach()を使ってweighted_sum_item_embeddingsからの勾配伝播は行わない
        # サーバーがこの目標値に向けて自身を更新するイメージ
        loss = loss_fn(self.global_item_embedding.weight, weighted_sum_item_embeddings.detach())
        
        loss.backward() # 勾配を計算
        self.global_item_optimizer.step() # パラメータを更新
        
        return self.global_item_embedding.weight.data # 更新後のデータを返す

# データセットの準備とクライアントへの分割 (1クライアント1ユーザー)
num_users = 100
num_items = 50
num_clients = num_users 

user_texts = {i: f"This user likes movies about {i % 5} and enjoys {i % 3}." for i in range(num_users)}

interactions_list = []
user_interaction_history = defaultdict(list) 

for u_id in range(num_users):
    for i_id in range(num_items):
        if np.random.rand() > 0.7:
            interactions_list.append([u_id, i_id, 1])
            user_interaction_history[u_id].append(i_id) 
        else:
            interactions_list.append([u_id, i_id, 0])

interactions = torch.tensor(interactions_list, dtype=torch.float32)

client_user_map = {} 
client_datasets = {}
client_original_data_sizes = {} 
for u_id in range(num_users):
    client_id = u_id 
    client_user_map[client_id] = u_id 
    
    client_interactions_indices = [i for i, (u, _, _) in enumerate(interactions_list) if u == u_id]
    
    if not client_interactions_indices:
        print(f"Warning: User {u_id} has no interactions. Client {client_id} will have an empty dataset.")
        client_subset = TensorDataset(
            torch.empty(0, dtype=torch.long), 
            torch.empty(0, dtype=torch.long), 
            torch.empty(0, dtype=torch.float32)
        )
        client_original_data_sizes[client_id] = 0
    else:
        user_interaction_data_for_client = []
        for idx in client_interactions_indices:
            u_id_data, i_id_data, label_data = interactions_list[idx]
            user_interaction_data_for_client.append((u_id_data, i_id_data, label_data))

        users_tensor = torch.tensor([d[0] for d in user_interaction_data_for_client], dtype=torch.long)
        items_tensor = torch.tensor([d[1] for d in user_interaction_data_for_client], dtype=torch.long)
        labels_tensor = torch.tensor([d[2] for d in user_interaction_data_for_client], dtype=torch.float32)

        client_subset = TensorDataset(users_tensor, items_tensor, labels_tensor)
        client_original_data_sizes[client_id] = len(client_subset)
    
    client_datasets[client_id] = DataLoader(client_subset, batch_size=min(32, max(1, len(client_subset))), shuffle=True)

print(f"Number of users: {num_users}")
print(f"Number of items: {num_items}")
print(f"Total interactions: {len(interactions)}")
print(f"Number of clients (1 client per user): {num_clients}")


# モデルのハイパーパラメータ
item_embedding_dim = 32
joint_embedding_output_dim = 32 

# サーバーのインスタンス化
server = Server(num_users, num_items, item_embedding_dim, joint_embedding_output_dim)

# 各クライアントのモデルを辞書で保持
client_models = {}
client_optimizers = {}
for client_id in range(num_clients):
    client_models[client_id] = ClientModel(
        num_items,
        item_embedding_dim,
        plm_model,
        plm_embedding_dim,
        joint_embedding_output_dim
    )
    # NOTE:
    # クライアントごとに最適化するパラメータを設定
    # ここでは、user_joint_embedding_linear, local_item_embedding, prediction_layer が対象
    # 単純にoptim.Adam(params = client_models[client_id].parameters(), lr=0.001)とすると、
    # PLMも学習可能パラメータとなってしまうので、
    # PLMのパラメータを除外したパラメータのみを取得してから、設定する.
    trainable_params = [
        p for name, p in client_models[client_id].named_parameters()
        if not name.startswith('plm_model.')
    ]

    client_optimizers[client_id] = optim.Adam(
        params=trainable_params,
        lr=0.001
    )

# 学習ループ (フェデレーテッド学習ラウンド)
num_communication_rounds = 10
local_epochs = 1 

for round_num in range(num_communication_rounds):
    print(f"\n--- Communication Round {round_num + 1}/{num_communication_rounds} ---")
    
    # サーバーからグローバルアイテム埋め込みをクライアントに配布
    for client_id in range(num_clients):
        client_models[client_id].local_item_embedding.weight.data.copy_(server.global_item_embedding.weight.data)

    user_linear_weights_for_graph = {} 
    user_local_item_weights_to_server = {} 
    client_reported_data_sizes = {} 
    
    # クライアントのローカル学習
    for client_id in range(num_clients):
        model = client_models[client_id]
        optimizer = client_optimizers[client_id]
        dataloader = client_datasets[client_id]
        
        model.train()
        local_loss = 0
        
        current_user_id = client_user_map[client_id] 
        
        client_reported_data_sizes[current_user_id] = client_original_data_sizes[client_id]

        if len(dataloader.dataset) == 0:
            print(f"  Client {client_id} (User {current_user_id}) has no interactions, skipping local training.")
            user_linear_weights_for_graph[current_user_id] = model.user_joint_embedding_linear.weight.data.clone().flatten()
            user_local_item_weights_to_server[current_user_id] = model.local_item_embedding.weight.data.clone()
            continue

        for epoch in range(local_epochs):
            for user_ids_batch, item_ids_batch, labels_batch in dataloader:
                assert torch.all(user_ids_batch == current_user_id) 
                current_user_texts = [user_texts[uid.item()] for uid in user_ids_batch]
                
                optimizer.zero_grad()
                predictions, user_joint_embedding_linear_weight, local_item_embedding_weight = model(
                    user_ids_batch, item_ids_batch, current_user_texts
                )
                
                loss = nn.BCEWithLogitsLoss()(predictions.squeeze(), labels_batch)
                
                lambda_reg = 0.01 
                
                # Note: `local_item_embedding_weight` は `requires_grad=True` ですが、
                # `server.global_item_embedding.weight.data` は `requires_grad=False` です。
                # このため、`regularization_term` の勾配は `local_item_embedding_weight` にのみ伝播します。
                # これは意図された動作です。
                regularization_term = torch.mean(
                    (local_item_embedding_weight - server.global_item_embedding.weight.data)**2
                )
                
                loss = loss + lambda_reg * regularization_term
                
                loss.backward()
                optimizer.step()
                local_loss += loss.item()

        user_linear_weights_for_graph[current_user_id] = user_joint_embedding_linear_weight.data.clone().flatten()
        user_local_item_weights_to_server[current_user_id] = local_item_embedding_weight.data.clone()

        print(f"  Client {client_id} (User {current_user_id}) local loss: {local_loss / len(dataloader):.4f}")

    # サーバーでの処理
    user_graph_adj, sorted_user_ids_for_graph = server.build_user_relationship_graph(
        user_linear_weights_for_graph
    )
    
    # グローバルアイテム埋め込みの更新を、サーバーのオプティマイザで行う関数を呼び出す
    server.update_global_item_embedding(
        user_local_item_weights_to_server, 
        user_graph_adj, 
        sorted_user_ids_for_graph,
        client_reported_data_sizes 
    )

    print(f"Round {round_num + 1} completed. Global item embeddings updated.")

print("Federated training completed.")

/Users/kentaro.suzuki/Library/Caches/pypoetry/virtualenvs/project-UD7q69fU-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PLM embedding dimension: 384
Number of users: 100
Number of items: 50
Total interactions: 5000
Number of clients (1 client per user): 100

--- Communication Round 1/10 ---
  Client 0 (User 0) local loss: 0.7839
  Client 1 (User 1) local loss: 0.8479
  Client 2 (User 2) local loss: 0.8426
  Client 3 (User 3) local loss: 0.7891
  Client 4 (User 4) local loss: 0.8219
  Client 5 (User 5) local loss: 0.8059
  Client 6 (User 6) local loss: 0.8673
  Client 7 (User 7) local loss: 0.8535
  Client 8 (User 8) local loss: 0.8148
  Client 9 (User 9) local loss: 0.7695
  Client 10 (User 10) local loss: 0.9366
  Client 11 (User 11) local loss: 0.8265
  Client 12 (User 12) local loss: 0.8255
  Client 13 (User 13) local loss: 0.8200
  Client 14 (User 14) local loss: 0.8185
  Client 15 (User 15) local loss: 0.7798
  Client 16 (User 16) local loss: 0.8306
  Client 17 (User 17) local loss: 0.8093
  Client 18 (User 18) local loss: 0.8758
  Client 19 (User 19) local loss: 0.7985
  Client 20 (User 20) local 